In [1]:
import json
import pandas as pd
import os
import shutil

DATA_DIR = "../data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")

store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도  # 위치기반 음식점 추천할 때 사용
    "longitude",  # 음식점 경도  # 위치기반 음식점 추천할 때 사용
    "category",  # 음식점 카테고리
    "review_cnt",  # 리뷰 갯수
)

menu_colums = (
    "store",  # 음식점 고유번호
    "menu",  # 메뉴 이름
    "price",  # 메뉴 가격
)

hour_colums = (
    "type",  # 영업시간 종류
    "week_type",  # 주단위 종류
    "store",  # 음식점 고유번호
    "mon",  # 월요일 포함유무
    "tue",  # 화요일 포함유무
    "wed",  # 수요일 포함유무
    "thu",  # 목요일 포함유무
    "fri",  # 금요일 포함유무
    "sat",  # 토요일 포함유무
    "sun",  # 일요일 포함유무
    "start_time",  # 시작시간
    "end_time",  # 종료시간
    "etc",  # 기타
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

user_colums = (
    "id",  # 유저 고유번호
    "gender",  # 성별
    "born_year"  # 태어난 해
)


def import_data(data_path=DATA_FILE):
    """
    Req. 1-1-1 음식점 데이터 파일을 읽어서 Pandas DataFrame 형태로 저장합니다
    """

    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    stores = []  # 음식점 테이블
    menus = []  # 메뉴 테이블
    hours = []  # 음식점 시간 테이블
    reviews = []  # 리뷰 테이블
    users = []  # 유저 테이블

    for d in data:

        categories = [c["category"] for c in d["category_list"]]
        # review_cnt = 
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                "|".join(categories),
                d["review_cnt"],
            ]
        )

        for menu in d["menu_list"]:
            menus.append(
                [d["id"], menu["menu"], menu["price"]]
            )
        
        for hour in d["bhour_list"]:
            hours.append(
                [
                    hour["type"], hour["week_type"], d["id"],
                    hour["mon"], hour["tue"], hour["wed"], hour["thu"],
                    hour["fri"], hour["sat"], hour["sun"],
                    hour["start_time"], hour["end_time"], hour["etc"]
                ]
            )
  
        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
            )

            users.append(
                [u["id"], u["gender"], u["born_year"]]
            )

    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_colums)
    hour_frame = pd.DataFrame(data=hours, columns=hour_colums)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    user_frame = pd.DataFrame(data=users, columns=user_colums)

    return {"stores": store_frame, "menus": menu_frame, "hours": hour_frame,
            "reviews": review_frame, "users": user_frame}


def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)


def load_dataframes():
    return pd.read_pickle(DUMP_FILE)


def main():

    print("[*] Parsing data...")
    data = import_data()
    print("[+] Done")

    print("[*] Dumping data...")
    dump_dataframes(data)
    print("[+] Done\n")

    data = load_dataframes()

    term_w = shutil.get_terminal_size()[0] - 1
    separater = "-" * term_w

    print("[음식점]")
    print(f"{separater}\n")
    print(data["stores"].head())
    print(f"\n{separater}\n\n")

    print("[메뉴]")
    print(f"{separater}\n")
    print(data["menus"].head())
    print(f"\n{separater}\n\n")

    print("[영업시간]")
    print(f"{separater}\n")
    print(data["hours"].head())
    print(f"\n{separater}\n\n")

    print("[리뷰]")
    print(f"{separater}\n")
    print(data["reviews"].head())
    print(f"\n{separater}\n\n")

    print("[유저]")
    print(f"{separater}\n")
    print(data["users"].head())
    print(f"\n{separater}\n\n")


if __name__ == "__main__":
    main()


[*] Parsing data...
[+] Done
[*] Dumping data...
[+] Done

[음식점]
-----------------------------------------------------------------------------------------------------------------------

   id     store_name branch  area            tel                address  \
0   1           Agal   None    홍대  010-6689-5886   서울특별시 마포구 동교동 170-13   
1   2         Assisy   None    광주   062-367-0700    광주광역시 서구 농성동 631-33   
2   3  Battered Sole   None   이태원    02-749-6867   서울특별시 용산구 이태원동 118-9   
3   4      Chakyoung   None  달맞이길   051-756-5566  부산광역시 해운대구 중2동 1509-5   
4   5       Delabobo   None   발산역   02-2667-9854      서울특별시 강서구 등촌동 689   

    latitude   longitude   category  review_cnt  
0  37.556862  126.926666   아구찜|포장마차           0  
1  35.150746  126.890062         카페           0  
2  37.535032  126.991664    피쉬앤칩스|펍           0  
3  35.158587  129.175004  레스토랑|카프레제           0  
4  37.559904  126.840512  디저트카페|디저트           0  

--------------------------------------------------------------

In [2]:
from parse import load_dataframes
import pandas as pd
import shutil

def get_most_active_users(dataframes):
    """
    Req. 1-2-4 가장 많은 리뷰를 작성한 `n`명의 유저를 정렬하여 리턴합니다.
    """
    # users와 reviews를 병합
    # 이때 users_reviews가 아닌 reviews_users로 병합하면 똑같은 리뷰가 엄청 여러개 나옴 why??????????????????
    # user = dataframes["users"]
    
    users_reviews = pd.merge(
        dataframes["users"], dataframes["reviews"], left_on="id", right_on="user"
    )

    # user
    user_group = users_reviews.groupby(['user'])

    # user_by_reviews_cnt= users_reviews.groupby(["user"]).count()

    return user_group.head(n=15).reset_index()


def main():
    data = load_dataframes()
    term_w = shutil.get_terminal_size()[0] - 1
    separater = "-" * term_w
    
    users_most_review = get_most_active_users(data)

    print("[가장 많은 리뷰를 작성한 유저]")
    print(f"{separater}\n")
    print(users_most_review)
    
    

if __name__ == "__main__":
    main()


[가장 많은 리뷰를 작성한 유저]
-----------------------------------------------------------------------------------------------------------------------

         index    id_x gender born_year  id_y   store    user  score  \
0            0   68632      남      1990     1      15   68632      5   
1            1   68632      남      1990   326    1216   68632      5   
2            2   68632      남      1990  1413    8756   68632      5   
3            3   68632      남      1990  1528    9460   68632      5   
4            4   68632      남      1990  4681   13745   68632      5   
...        ...     ...    ...       ...   ...     ...     ...    ...   
85896  5394305  171391      남      1993  2367  360464  171391      4   
85897  5394306  518267      여      1995  2376  360465  518267      5   
85898  5394307  280855      남      1988  2382  360467  280855      5   
85899  5394308  456281      남      1967  2393  360486  456281      4   
85900  5394309   17371      여      1986  2401  360499   17371      5